#STM Analysis
### Date: 2016-11-??
### Author: Gerardo Salazar

##Company Summary
###Excerpt From [Reuters](http://www.reuters.com/finance/stocks/companyProfile?rpc=66&symbol=STM)
STMicroelectronics N.V., incorporated on May 21, 1987, is a global semiconductor company. The Company is focused on Smart Driving, enabled by digitalization and electrification of the car, and the Internet of Things, including portable and wearable systems, as well as smart home, city and industry applications. The Company's segments include Automotive and Discrete Group (ADG), Analog and MEMS Group (AMG), Microcontrollers and Digital ICs Group (MDG), and Others.

##Company Statistics (Last 4 Years)

In [ ]:
import pandas
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import matplotlib

from quantopian.pipeline.data import morningstar


fundamentals = init_fundamentals()

fund_df = get_fundamentals(
            query(
                fundamentals.valuation.market_cap,
                fundamentals.valuation.enterprise_value,
                fundamentals.income_statement.total_revenue,
                fundamentals.income_statement.total_expenses
            ), '2017-06-28', '4q')

STM_data = fund_df.minor_xs(symbols('STM'))
STM_data['market_cap'] = STM_data['market_cap'].apply(np.int64)
STM_data['enterprise_value'] = STM_data['enterprise_value'].apply(np.int64)
STM_data['total_revenue'] = STM_data['total_revenue'].apply(np.int64)
STM_data['total_expenses'] = STM_data['total_expenses'].apply(np.int64)

In [ ]:
STM_data.plot(title='STM Fundamentals', logy=True)

* For the last 3 years, __STM__'s market cap has been stuck in the __\$6.0B - $8.5B__ range. 
* Enterprise value has shot up __56.448271223%__ since __2016-11-04__. 
* Revenue increased __3.505843072%__ and expenses increased __1.01171646163%__ from the previous quarter.
* Expenses have been on a downwards trend since __2013__, but is now in an upwards trend since the start of __2016__
* Total revenue is also in an upwards trend

##Historical Analysis


In [ ]:
history_ev = fund_df.loc['enterprise_value']
security_interest = []
security_index = []

security_statistics = []

# The way this loop works is that it loops the enterprise_value column. Since market_cap and enterprise_value
# have the same amount of columns, we can reference any other value we want through that. Because of that, we
# only need to have 1 loop instead 
for ev in history_ev:
    # security_ev stores the entire history of 1 security. Every time it iterates, it jumps to
    # another security
    security_ev = history_ev[ev]
    
    # Start another for loop to iterate through the data each security holds
    for index, ev_security_data in enumerate(security_ev):
        if np.isnan(ev_security_data) or index == 0 or index > 14 or np.isnan(security_ev.values[-1]):
            # we don't want incomplete data in this dataset
            # we also can't compare the first value to the one before it, so let's skip
            # we also don't want values from the last 2 quarters so we can see how well these stocks did
            # we also don't want stocks that have NaN values 2 quarters after the current
            continue
            
        previous_ev = security_ev.values[index - 1]
        current_ev = security_ev.values[index]
        
        previous_cap = fund_df.loc['market_cap'][ev].values[index - 1]
        current_cap = fund_df.loc['market_cap'][ev].values[index]
        
        ev_delta_pct = (current_ev - previous_ev) / previous_ev
        cap_delta_pct = (current_cap - previous_cap) / previous_cap
        
        if (cap_delta_pct > 0.52 and cap_delta_pct < 0.58) and (ev_delta_pct > 0.52 and ev_delta_pct < 0.58):
            security_interest.append(ev)
            security_index.append(index - 1)
            break

for sec_index, security in enumerate(security_interest):
    security_history = fund_df.minor_xs(security)
    sec_mc = security_history['market_cap']
    
    temp_mean_list = []
    lowest_point = None
    highest_point = None
    
    for index, mc_history in enumerate(sec_mc):
        if index == security_index[sec_index]:
            starting_point = security_history.iloc[index]['market_cap']

            sec_plot = security_history.plot(title=security.symbol, logy=True)
            startcoords = (security_history.index[index], security_history.iloc[index]['market_cap'] * 1.05)
            endcoords = (security_history.index[index + 1], security_history.iloc[index + 1]['market_cap'] * 1.05)
            dateasnum = matplotlib.dates.date2num(security_history.index[index])
            
            sec_plot.annotate(
                'Start',
                xy=startcoords,
                xytext=(dateasnum + 30, startcoords[1]*1.3),
                arrowprops=dict(arrowstyle="fancy", connectionstyle="arc3, rad=0.3", fc="k"),
                bbox=dict(boxstyle="round", fc="#DDDDDD")
            )
            sec_plot.annotate(
                'End',
                xy=endcoords,
                xytext=(dateasnum + 30, endcoords[1]*1.3),
                arrowprops=dict(arrowstyle="fancy", connectionstyle="arc3, rad=0.3", fc="k"),
                bbox=dict(boxstyle="round", fc="#DDDDDD")
            )
            
            sec_plot
            
            for quarter in range(0, len(sec_mc) - index):
                ending_point = None
                temp_mean_list.append(security_history.iloc[index + quarter]['market_cap'])
                if quarter == 0:
                    lowest_point = temp_mean_list[quarter]
                    highest_point = temp_mean_list[quarter]
                    
                if quarter >= 1:
                    if temp_mean_list[quarter] > highest_point:
                        highest_point = temp_mean_list[quarter]
                    
                    if temp_mean_list[quarter] < lowest_point:
                        lowest_point = temp_mean_list[quarter]

                if quarter == (len(sec_mc) - index) - 1:
                    ending_point = security_history.iloc[-1]['market_cap']
                    
                    security_statistics.append(
                        [
                            security,
                            np.mean(temp_mean_list),
                            lowest_point,
                            highest_point,
                            starting_point,
                            ending_point
                        ]
                    )
                    break
                    
                # we will analyze 6 quarters after the growth period and append it to our growth variable
                # we will store the variables in the following order:
                # security, mean, lowest, max, start, end

###Quantitative Analysis and Finding Out If Works

I have created a lot of charts that have similar patterns to the ones seen in __STM__. <br />
There you can see the growth of these companies after the period that matches __STM__.

In [ ]:
security_statistics

In [ ]:
win_count = 0
lose_count = 0

for index, stat_vals in enumerate(security_statistics):
    if stat_vals[4] < stat_vals[5]:
        win_count += 1
    else:
        lose_count += 1

print(len(security_statistics))
print(win_count)
print(lose_count)
print("Win Percentage is: {}".format((float(win_count) / float(len(security_statistics))) * 100.0))
print("Lose Percentage is: {}".format((float(lose_count) / float(len(security_statistics))) * 100.0)